In [3]:
!pip install selenium webdriver-manager pandas charset_normalizer
!pip install openpyxl # For Excel support in pandas
# resumir la salida de las celdas


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Alvaro\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\Alvaro\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [7]:
"""
scrape_dge_dengue_tendencias_full.py (v14 - NameError Fix)

- Se corrige un error 'NameError' moviendo todas las definiciones de funciones de ayuda
  al principio del script, asegurando que estén definidas antes de ser utilizadas.
- El resto de la lógica flexible y robusta de la v13 se mantiene intacta.
- Requiere 'pip install openpyxl'.
"""

import os
import time
import glob
import logging
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# ==============================================================================
# --- DEFINICIÓN DE FUNCIONES DE AYUDA (MOVIDAS AL PRINCIPIO) ---
# ==============================================================================

def normalize_filename(s: str):
    """Limpia una cadena para que sea un nombre de archivo/carpeta seguro."""
    if s is None: return ""
    return "".join(c if c.isalnum() else "_" for c in s).replace("__", "_").strip("_")

def make_driver(download_dir, headless):
    opts = webdriver.ChromeOptions()
    prefs = {"download.default_directory": download_dir, "download.prompt_for_download": False, "plugins.always_open_xlsx_externally": True}
    opts.add_experimental_option("prefs", prefs)
    opts.add_argument("--disable-blink-features=AutomationControlled"); opts.add_argument("--start-maximized")
    if headless: opts.add_argument("--headless=new"); opts.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def wait_for_js_idle(driver, pause=0.8, max_tries=60):
    for _ in range(max_tries):
        time.sleep(pause);
        try:
            if not any(o.is_displayed() for o in driver.find_elements(By.CSS_SELECTOR, "div.shiny-load-requests-waiting, div.shiny-busy")): return True
        except Exception: pass
    logging.warning("wait_for_js_idle alcanzó el máximo de intentos."); return False

def ensure_download_completed(download_dir, before_files, timeout=180):
    start = time.time()
    while time.time() - start < timeout:
        new = set(os.listdir(download_dir)) - before_files
        if new and not any(f.endswith((".crdownload", ".tmp")) for f in new):
            time.sleep(2); return list(new)
        time.sleep(1)
    return []

def expand_panel(driver, wait, panel_text):
    try:
        xpath = f"//button[contains(@class, 'accordion-button') and .//div[normalize-space()='{panel_text}']]"
        panel_button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
        if panel_button.get_attribute('aria-expanded') == 'false':
            logging.info(f"Expandiendo panel '{panel_text}'...")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", panel_button)
            time.sleep(0.5); panel_button.click(); time.sleep(1.5)
        return True
    except Exception as e:
        logging.error(f"No se pudo expandir el panel '{panel_text}': {e}"); return False

def set_slider_to_full_range(driver, wait):
    logging.info("Ajustando el slider de período al rango completo...")
    try:
        slider_input = wait.until(EC.presence_of_element_located((By.ID, "tendencias-sel_ano")))
        min_val, max_val = slider_input.get_attribute("data-min"), slider_input.get_attribute("data-max")
        script = f"""$("#tendencias-sel_ano").data("ionRangeSlider").update({{ from: {min_val}, to: {max_val} }});"""
        driver.execute_script(script); logging.info(f"Slider ajustado a {min_val}-{max_val} con JavaScript."); return True
    except Exception as e:
        logging.error(f"Falló el ajuste del slider: {e}"); return False

def find_interactive_element_for_label(driver, wait, label_text):
    try:
        xpath = f"//label[normalize-space()='{label_text}']/following-sibling::div//input"
        return wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
    except Exception: logging.warning(f"No se pudo encontrar un elemento para la etiqueta '{label_text}'."); return None

def select_option_in_selectize(driver, input_element, option_text, retries=3):
    if not input_element: return False
    for attempt in range(retries):
        try:
            parent = input_element.find_element(By.XPATH, "./ancestor::div[contains(@class, 'selectize-input')]")
            driver.execute_script("arguments[0].click();", parent)
            time.sleep(0.5); input_element.send_keys(Keys.CONTROL + "a"); input_element.send_keys(Keys.BACK_SPACE)
            time.sleep(0.3); input_element.send_keys(option_text); time.sleep(1.5)
            option_xpath = f"//div[contains(@class,'selectize-dropdown-content')]//div[normalize-space()='{option_text}']"
            option_to_click = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, option_xpath)))
            option_to_click.click(); wait_for_js_idle(driver); return True
        except Exception:
            logging.debug(f"Intento {attempt + 1}/{retries} para seleccionar '{option_text}' falló. Reintentando...")
            ActionChains(driver).send_keys(Keys.ESCAPE).perform(); time.sleep(1)
    logging.error(f"Error final seleccionando '{option_text}' después de {retries} intentos."); return False

def get_options_from_selectize(driver, input_element, exclude=None):
    options = []; exclude_lower = [e.lower() for e in (exclude or [])]
    if not input_element: return options
    try:
        parent = input_element.find_element(By.XPATH, "./ancestor::div[contains(@class, 'selectize-input')]")
        driver.execute_script("arguments[0].click();", parent); time.sleep(1)
        opts = driver.find_elements(By.XPATH, "//div[contains(@class, 'selectize-dropdown') and not(contains(@style,'display: none'))]//div[@class='option']")
        for o in opts:
            t = o.text.strip()
            if t and t.lower() not in ("seleccione", "todos", "...", "nacional") and t.lower() not in exclude_lower: options.append(t)
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    except Exception as e:
        logging.error(f"Error obteniendo opciones del selectize: {e}")
    return list(dict.fromkeys(options))


# ==============================================================================
# ---------- CONFIGURACIÓN DE BÚSQUEDA (¡MODIFICAR SOLO ESTA SECCIÓN!) ----------
# ==============================================================================
# 1. Escribe el nombre de la enfermedad EXACTAMENTE como aparece en el menú.
#    Ejemplos: "Dengue", "Leptospirosis", "Ofidismo", "Leishmaniasis",
#              "Malaria", "Febriles", "Enfermedad diarreica aguda", "Infeccion respiratoria aguda",
#              "Neumonia", "SOB/Asma"
# 2. Escribe el tipo de diagnóstico a buscar.
#    - Si la opción existe, la seleccionará.
#    - Si NO existe, usará el valor por defecto de la página y continuará.
#    - Pon None (sin comillas) para saltar este paso y usar siempre el valor por defecto.
#    Ejemplos: "TOTAL", "CONFIRMADOS", "PROBABLES", None
ENFERMEDAD_BUSCADA = "Dengue"
TIPO_DIAGNOSTICO_BUSCADO = "TOTAL"
BASE_DOWNLOAD_DIR = r'D:\UNIDAD D\UNIVERSIDAD\2025-2\Modelos Fisiológicos\Modelos_fisio\datasets\CDC_datasets'
BASE_URL = "https://app7.dge.gob.pe/maps2/shiny_observatorio_web/"
WAIT = 30
HEADLESS = False
LOGLEVEL = logging.INFO
# ==============================================================================

# --- CONSTRUCCIÓN DE RUTAS Y CONFIGURACIÓN (AHORA FUNCIONARÁ) ---
DOWNLOAD_DIR = os.path.join(BASE_DOWNLOAD_DIR, normalize_filename(ENFERMEDAD_BUSCADA))
try:
    os.makedirs(DOWNLOAD_DIR, exist_ok=True)
except OSError as e:
    logging.error(f"Error al crear el directorio de descarga '{DOWNLOAD_DIR}': {e}"); exit()
logging.basicConfig(level=LOGLEVEL, format="%(asctime)s - %(levelname)s - %(message)s")


def main():
    driver = make_driver(download_dir=DOWNLOAD_DIR, headless=HEADLESS)
    wait = WebDriverWait(driver, WAIT)
    try:
        logging.info(f"INICIANDO PROCESO PARA LA ENFERMEDAD: '{ENFERMEDAD_BUSCADA}'")
        logging.info(f"Los archivos se guardarán en: {DOWNLOAD_DIR}")
        driver.get(BASE_URL); wait_for_js_idle(driver)

        wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@data-value='tendencias']"))).click()
        time.sleep(3); wait_for_js_idle(driver)

        logging.info(f"Configurando filtros para '{ENFERMEDAD_BUSCADA}'...")
        if not select_option_in_selectize(driver, find_interactive_element_for_label(driver, wait, "Enfermedad/episodio"), ENFERMEDAD_BUSCADA):
            logging.error(f"No se pudo seleccionar la enfermedad '{ENFERMEDAD_BUSCADA}'. Abortando."); return
        
        if TIPO_DIAGNOSTICO_BUSCADO:
            logging.info(f"Intentando seleccionar el Tipo de diagnóstico: '{TIPO_DIAGNOSTICO_BUSCADO}'...")
            tipo_input = find_interactive_element_for_label(driver, wait, "Tipo de diagnóstico o forma clínica")
            if tipo_input:
                if not select_option_in_selectize(driver, tipo_input, TIPO_DIAGNOSTICO_BUSCADO):
                    logging.warning(f"No se pudo seleccionar el tipo '{TIPO_DIAGNOSTICO_BUSCADO}'. Se usará la opción por defecto.")
            else:
                logging.warning("No se encontró el campo 'Tipo de diagnóstico'. Se usará el valor por defecto.")
        else:
            logging.info("No se especificó Tipo de diagnóstico. Se usará la opción por defecto.")

        if not expand_panel(driver, wait, "Periodo de interés") or not set_slider_to_full_range(driver, wait): return
        if not expand_panel(driver, wait, "Lugar de interés"): return
        
        dep_input = find_interactive_element_for_label(driver, wait, "Departamento")
        departamentos = get_options_from_selectize(driver, dep_input)
        if not departamentos: logging.error("No se pudieron obtener departamentos."); return
        logging.info(f"Se descargará un archivo por cada uno de los {len(departamentos)} departamentos.")

        normalized_enfermedad = normalize_filename(ENFERMEDAD_BUSCADA)

        for dep in departamentos:
            logging.info(f"=== Procesando Departamento: {dep.upper()} ===");
            if not select_option_in_selectize(driver, dep_input, dep):
                logging.warning(f"No se pudo seleccionar {dep}, saltando al siguiente."); continue
            try:
                wait.until(EC.element_to_be_clickable((By.ID, "tendencias-btn_zona"))).click()
                wait_for_js_idle(driver, pause=2, max_tries=90)
                download_link = wait.until(EC.element_to_be_clickable((By.ID, "tendencias-down_climasalud")))
                before = set(os.listdir(DOWNLOAD_DIR)); download_link.click()
                downloaded = ensure_download_completed(DOWNLOAD_DIR, before)
                if downloaded:
                    old_path = os.path.join(DOWNLOAD_DIR, downloaded[0])
                    new_name = f"{normalized_enfermedad}_completo_{normalize_filename(dep)}.xlsx"
                    new_path = os.path.join(DOWNLOAD_DIR, new_name)
                    if os.path.exists(new_path): os.remove(new_path)
                    os.rename(old_path, new_path)
                    logging.info(f"  -> ¡Éxito! Archivo guardado como: {new_name}")
                else:
                    logging.warning("  -> No se detectó archivo descargado (timeout).")
            except Exception as e:
                logging.error(f"  -> Falló la secuencia de descarga para {dep}: {e}")

        logging.info("="*50 + f"\nPROCESO DE DESCARGA PARA '{ENFERMEDAD_BUSCADA}' FINALIZADO. CONSOLIDANDO...\n" + "="*50)
        all_xlsx = glob.glob(os.path.join(DOWNLOAD_DIR, "*.xlsx"))
        if not all_xlsx: logging.error("No se encontraron archivos .xlsx para consolidar."); return

        df_list = []
        for f_path in all_xlsx:
            try:
                xls = pd.ExcelFile(f_path)
                target_sheet = next((s for s in xls.sheet_names if s.lower() == 'distrito'), None)
                if target_sheet:
                    df = pd.read_excel(xls, sheet_name=target_sheet)
                    df['origen_archivo'] = os.path.basename(f_path); df_list.append(df)
                else:
                    logging.warning(f"No se encontró hoja 'Distrito' en {os.path.basename(f_path)}. Hojas: {xls.sheet_names}")
            except Exception as e:
                logging.warning(f"No se pudo procesar {os.path.basename(f_path)}: {e}")
        
        if not df_list: logging.error("No se pudo leer datos de ningún archivo Excel."); return
        final_df = pd.concat(df_list, ignore_index=True)
        output_path = os.path.join(BASE_DOWNLOAD_DIR, f"{normalized_enfermedad}_consolidado_distrital_final.csv")
        final_df.to_csv(output_path, index=False)
        logging.info("="*50 + f"\n¡PROCESO COMPLETADO CON ÉXITO!\nArchivo consolidado en: {output_path}")
        logging.info(f"El dataset final tiene {final_df.shape[0]} filas y {final_df.shape[1]} columnas.\n" + "="*50)

    except Exception:
        logging.error("Ocurrió un error fatal en el proceso principal.", exc_info=True)
    finally:
        driver.quit()

if __name__ == "__main__":
    main()

2025-09-17 02:28:19,756 - INFO - ====== WebDriver manager ======
2025-09-17 02:28:20,768 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-17 02:28:20,860 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-17 02:28:20,923 - INFO - Driver [C:\Users\Alvaro\.wdm\drivers\chromedriver\win64\140.0.7339.82\chromedriver-win32/chromedriver.exe] found in cache
2025-09-17 02:28:22,000 - INFO - INICIANDO PROCESO PARA LA ENFERMEDAD: 'Dengue'
2025-09-17 02:28:22,001 - INFO - Los archivos se guardarán en: D:\UNIDAD D\UNIVERSIDAD\2025-2\Modelos Fisiológicos\Modelos_fisio\datasets\CDC_datasets\Dengue
2025-09-17 02:28:29,352 - INFO - Configurando filtros para 'Dengue'...
2025-09-17 02:28:32,810 - INFO - Intentando seleccionar el Tipo de diagnóstico: 'TOTAL'...
2025-09-17 02:28:36,227 - INFO - Expandiendo panel 'Periodo de interés'...
2025-09-17 02:28:38,318 - INFO - Ajustando el slider de período al rango completo...
2025-09-17 02:28:38,355 - INFO - Slider ajustado a 201